# WZE-UAV Image Classification using Deep Learning

In [1]:
import os
import gc
import glob
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold

In [2]:
import torch
import torchvision.transforms as T
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from mlxtend.plotting import plot_confusion_matrix

In [3]:
import wze_uav.data_loader as data_loader
import wze_uav.models as models
from wze_uav.engine import *
from wze_uav.utils2 import *
#from wze_uav.log_writer import create_writer
from wze_uav.datasplit import *
from efficientnet import model_effnet #for custom effnet with n_channels input
import wandb

#### Get PyTorch version

In [4]:
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

torch version: 1.13.1+cu116
torchvision version: 0.14.1+cu116


#### Preparing device agnostic code 

In [5]:
# ensure device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# get index of currently selected device
print(f"Index of current divice: {torch.cuda.current_device()}")
# get number of GPUs available
print(f"Number of GPUs available: {torch.cuda.device_count()}")
# get the name of the device
print(f"GPU Model: {torch.cuda.get_device_name(0)}")

cuda
Index of current divice: 0
Number of GPUs available: 1
GPU Model: Quadro RTX 8000


#### Login to Weights & Biases to track results

In [6]:
# IMPORTANT: The proxy needs to be set in anaconda!
wandb.login()
#wandb.init(settings=wandb.Settings(start_method="thread"))
wandb.init(project='wze-uav', entity='simon-ecke')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: simon-ecke. Use `wandb login --relogin` to force relogin


#### Ensure reproducibility 

In [7]:
# for more information, see also: https://pytorch.org/docs/stable/notes/randomness.html

# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)
    # seed for numpy
    np.random.seed(seed)

set_seeds(42) 

# Set to true -> might speed up the process but should be set to False if reproducible results are desired
torch.backends.cudnn.benchmark = False


#### Define file directory

In [8]:
#####################################################################################
# 3 channel input (r-g-b)
#data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb"

# 4 channel input (r-g-b-nir)
data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb-nir"

# 5 channel input (r-g-b-re-nir)
#data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb-re-nir"
#####################################################################################

#### Get all file paths

In [9]:
fn_list = os.listdir(data_path)
path_list = []
# Iterate over all datafiles
for year in fn_list:
    year_dir = f'{data_path}\\{year}'
    for filename in os.listdir(year_dir):
        path = f'{year_dir}\\{filename}'
        path_list.append(path)

In [10]:
path_list

['D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10527_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10533_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10545_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10547_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10912_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr11032_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr11040_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr11044_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr11048_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_fin

#### Create unique hash IDs for every individual tree

In [11]:
hashID_dict = data_loader.get_unique_treeID(path_list)

Creating unique tree IDs...:   0%|          | 0/647 [00:00<?, ?it/s]

#### Import all imagery, labels and other features from hdf5 files

In [12]:
image_set, label_set, species_set, kkl_set, bk_set, hash_id = data_loader.hdf5_to_img_label(path_list,
                                                                                               hashID_dict,
                                                                                               load_sets=["images_masked"])

Processing hdf5 datasets:   0%|          | 0/647 [00:00<?, ?it/s]

#### Convert nbv to classes

In [13]:
label_set = nbv_to_sst_3classes(label_set)

#### Split data into a sub set and a test dataset

In [14]:
sub_image_set, sub_label_set, sub_hash_id, sub_species_set, test_image_set, test_label_set, test_hash_id, test_species_set = data_split(image_set, label_set, hash_id, species_set, test_size=0.1667, random_state=42, seed=2)


There are 7087 unique values within hash_id.
Number of True in sub_indices: 15659
Number of False in sub_indices: 3146
Number of True in test_indices: 1056
Number of False in test_indices: 17749
Check shapes:

Images sub dataset: (15659, 250, 250, 4)
Labels sub dataset: (15659, 1)

Images test dataset: (1056, 250, 250, 4)
Labels test dataset: (1056, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in sub dataset: 9232
Stressed trees in sub dataset: 6166
Dead trees in sub dataset: 261
Healthy trees in test dataset: 612
Stressed trees in test dataset: 429
Dead trees in test dataset: 15
Ratio health trees in test dataset: 0.06629116117850953
Ratio stressed trees in test dataset: 0.0695750891988323
Ratio dead trees in test dataset: 0.05747126436781609


#### Check if any hash ID is in both sub and test dataset

In [15]:
hash_set = set(sub_hash_id[:,0].flatten())
test_hash_set = set(test_hash_id[:,0].flatten())
intersection = hash_set.intersection(test_hash_set)
if intersection:
    print(f"Hash_id values in both train and test sets: {intersection}")
else:
    print("There are no hash_id values in both train and test datasets. The datasplit was successful")

There are no hash_id values in both train and test datasets. The datasplit was successful


#### Check feature distribution of the Test dataset

In [16]:
def count_occurrences(data, value):
    count = 0
    for item in data:
        if item == value:
            count += 1
    return count

print("Test dataset")
print(f"Test data healthy trees: {count_occurrences(test_label_set, 0)}")
print(f"Test data stressed trees: {count_occurrences(test_label_set, 1)}")
print(f"Test data dead trees: {count_occurrences(test_label_set, 2)}")
print(f"Test data pine trees: {count_occurrences(test_species_set, 134)}")
print(f"Test data spruces: {count_occurrences(test_species_set, 118)}")
print("-"*50)

print("Remaining dataset")
print(f"Remaining data healthy trees: {count_occurrences(sub_label_set, 0)}")
print(f"Remaining data stressed trees: {count_occurrences(sub_label_set, 1)}")
print(f"Remaining data dead trees: {count_occurrences(sub_label_set, 2)}")
print(f"Remaining data pine trees: {count_occurrences(sub_species_set, 134)}")
print(f"Remaining data spruces: {count_occurrences(sub_species_set, 118)}")
print("-"*50)

Test dataset
Test data healthy trees: 612
Test data stressed trees: 429
Test data dead trees: 15
Test data pine trees: 278
Test data spruces: 388
--------------------------------------------------
Remaining dataset
Remaining data healthy trees: 9232
Remaining data stressed trees: 6166
Remaining data dead trees: 261
Remaining data pine trees: 3960
Remaining data spruces: 5833
--------------------------------------------------


In [17]:
# train transform with augmentation. 
transform_train = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip(p=0.5), transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomRotation(degrees=[0,360])])

# test and val dataset transform without augmentation. 
transform = transforms.Compose([transforms.ToTensor()])

# class names need to fit the customDataset class used e.g. 3 classes -> use CustomDataset3Classes
#class_names = ['healthy', 'slightly_stressed', 'moderately_stressed', 'highly_stressed', 'dead']
#class_names = ['healthy', 'moderately_stressed', 'highly_stressed', 'dead']
class_names = ['healthy', 'stressed', 'dead']

# set seeds
g = torch.Generator()
g.manual_seed(42)
NUM_WORKERS=3 # should be changed, depending on the system used
batch_size=32

#### Define variables and parameters

In [18]:
# 1. Define number of epochs
epochs = 50
n_bands = sub_image_set[0].shape[2] # get number of bands

# 2. Define model
num_classes = len(class_names)
unfreeze = True # all layer weights get updated
dropout_rate = 0.5 #define dropout rate
model_name = "EffNet_b7_RGB-NIR_3classes_lr005_gamma05_dropout05"

# 3. Define loss, optimizer and learning rate scheduler
lr = 0.005 # define learning rate
min_lr = 1e-6 # minimum learning rate threshold
gamma = 0.5 # how fast the learning rate decreases per epoch (low number=faster decrease)
patience = 10

# 4. Create target folder name were to save the tensorboard event files
experiment_name = 'RGB-NIR_3classes_v3_lr005_gamma05_dropout05'
extra = "new_classes_lr005_gamma05_dropout05"

#----------------------------------------------------------------------
#torch.cuda.empty_cache()
print(f"Memory allocated: {torch.cuda.memory_allocated()} bytes") 



Memory allocated: 0 bytes


#### Run k-Fold cross-validation on EfficientNet

In [19]:
%%time

# Set the random seeds
set_seeds(42)

# Define the number of folds
num_folds = 5

# Create a KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

unique_values = np.unique(sub_hash_id[:,0]) # now there are less unique values left because we seperated the test dataset
num_unique = len(unique_values)
print(f"There are {num_unique} unique values within hash_id.\n")

# Loop over each fold
for fold, (train_ids, val_ids) in enumerate(kf.split(unique_values)):
    torch.cuda.empty_cache()
    
    # Initialize a new wandb run for this fold
    wandb.init(project='wze-uav', name=f"fold_{fold + 1}_{extra}")
    
    # 1. Split data into train and validation set
    # Get the training and testing data for this fold
    # Use np.isin() to create boolean arrays indicating which indices belong to train or test sets
    train_indices = np.isin(sub_hash_id[:,0], unique_values[train_ids])
    
    val_indices = np.zeros_like(train_indices)  # initialize to all False
    for hash_id_val in unique_values[val_ids]:
        # select one image ID randomly from either 2020 or 2021 or 2022 for each unique hash ID in the test set
        year = np.random.choice(['2020', '2021', '2022'])
        image_ids = sub_hash_id[(sub_hash_id[:,0] == hash_id_val) & (sub_hash_id[:,1] == year), 0]
    
        # mark the index corresponding to the selected image ID and hash ID as True in the test indices array
        val_indices[(sub_hash_id[:,0] == hash_id_val) & (sub_hash_id[:,1] == year) & (np.isin(sub_hash_id[:,0], image_ids))] = True 
    
    # Reshape boolean arrays to match shape of image_set and label_set
    train_indices = train_indices.reshape(-1, 1)
    val_indices = val_indices.reshape(-1, 1)
    
    # Select images and labels for train and validation sets
    train_image_set = sub_image_set[train_indices[:, 0]]
    train_label_set = sub_label_set[train_indices[:, 0]]
    train_hash_id = sub_hash_id[train_indices[:, 0]][:,0]
    train_species_set = sub_species_set[train_indices[:, 0]]
    val_image_set = sub_image_set[val_indices[:, 0]]
    val_label_set = sub_label_set[val_indices[:, 0]]
    val_hash_id = sub_hash_id[val_indices[:, 0]][:,0]
    val_species_set = sub_species_set[val_indices[:, 0]]
    # reshape 
    train_label_set = train_label_set.reshape(-1, 1)
    val_label_set = val_label_set.reshape(-1, 1)
    train_species_set = train_species_set.reshape(-1, 1)
    val_species_set = val_species_set.reshape(-1, 1)

         
    print("Check shapes:\n")
    print(f"Images train dataset: {train_image_set.shape}")
    print(f"Labels train dataset: {train_label_set.shape}\n")
    
    print(f"Images validation dataset: {val_image_set.shape}")
    print(f"Labels validation dataset: {val_label_set.shape}\n")
    print('-'*50)
    print (f"Check if the split was stratified: (random_state=42)")
    print(f"Healthy trees in train dataset: {np.count_nonzero(train_label_set == 0)}")
    print(f"Stressed trees in train dataset: {np.count_nonzero(train_label_set == 1)}")
    print(f"Dead trees in train dataset: {np.count_nonzero(train_label_set == 2)}")
    print(f"Healthy trees in validation dataset: {np.count_nonzero(val_label_set == 0)}")
    print(f"Stressed trees in validation dataset: {np.count_nonzero(val_label_set == 1)}")
    print(f"Dead trees in validation dataset: {np.count_nonzero(val_label_set == 2)}")
    print(f"Ratio health trees in validation dataset: {np.count_nonzero(val_label_set == 0)/np.count_nonzero(sub_label_set == 0)}")
    print(f"Ratio stressed trees in validation dataset: {np.count_nonzero(val_label_set == 1)/np.count_nonzero(sub_label_set == 1)}")
    print(f"Ratio dead trees in validation dataset: {np.count_nonzero(val_label_set == 2)/np.count_nonzero(sub_label_set == 2)}")
    print("-"*50)
   
    # 2. Create train and validation dataset. (choose custom dataset loader with 3 - 5 classes)
    print(f"\nCreating datasets for fold: {fold + 1}\n")
    train_dataset = data_loader.CustomDataset(data=train_image_set, labels=train_label_set, class_names=class_names, species = train_species_set,
                                                         transform=transform_train)
    
    val_dataset = data_loader.CustomDataset(data=val_image_set, labels=val_label_set, class_names=class_names,
                                                       species = val_species_set, transform=transform)
   
    # 3. Create train and validation dataloader
    # create sampler for oversampling of the minority classes
    sampler = data_loader.data_sampler(dataset=train_dataset, class_names=class_names)
    print(f"Creating dataloaders for fold: {fold +1}\n")
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, persistent_workers=True, pin_memory=True, num_workers=NUM_WORKERS, generator=g,
                              sampler=sampler, shuffle=False, drop_last=True) # shuffle false because of the sampler

    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, persistent_workers=True, pin_memory=True, num_workers=NUM_WORKERS, shuffle=False,
                             drop_last=True)
    
    model = model_effnet.EfficientNet.from_pretrained('efficientnet-b7', in_channels=n_bands, num_classes=num_classes, dropout_rate=dropout_rate)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    #lr_scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    lr_scheduler = ExponentialLR(optimizer, gamma=gamma)

    fold += 1
    print(f"\n[INFO] Fold number: {fold}")
    print(f"[INFO] Number of epochs: {epochs}")
    print(f"[INFO] Batch_size: {batch_size}")
    print(f"[INFO] Number of bands: {n_bands}")
    print(f"[INFO] Dropout rate: {dropout_rate}")
    print(f"[INFO] Gamma learning rate: {gamma}")
    print(f"[INFO] Memory allocated: {torch.cuda.memory_allocated()} bytes")
    # 4. Train model with k fold dataloaders and track experiments
    
    if fold == 1:
        fold1_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
       
    elif fold == 2:
        fold2_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    elif fold == 3:
        fold3_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    elif fold == 4:
        fold4_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    else:
        fold5_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    
    del train_indices, val_indices, train_image_set, train_label_set, train_hash_id, train_species_set, val_image_set, val_label_set, val_hash_id, val_species_set,
    train_dataset, val_dataset, sampler, train_dataloader, val_dataloader, model, loss_fn, optimizer, lr_scheduler
    
    #finish the wandb run
    wandb.finish()
    print("Deleting variables and emptying cache")
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Memory allocated: {torch.cuda.memory_allocated()} bytes")
    print("-"*50 + "\n")
    


There are 5905 unique values within hash_id.



Check shapes:

Images train dataset: (12564, 250, 250, 4)
Labels train dataset: (12564, 1)

Images validation dataset: (1043, 250, 250, 4)
Labels validation dataset: (1043, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in train dataset: 7401
Stressed trees in train dataset: 4959
Dead trees in train dataset: 204
Healthy trees in validation dataset: 618
Stressed trees in validation dataset: 404
Dead trees in validation dataset: 21
Ratio health trees in validation dataset: 0.06694107452339688
Ratio stressed trees in validation dataset: 0.06552059682127798
Ratio dead trees in validation dataset: 0.08045977011494253
--------------------------------------------------

Creating datasets for fold: 1

Creating dataloaders for fold: 1

Loaded pretrained weights for efficientnet-b7

[INFO] Fold number: 1
[INFO] Number of epochs: 50
[INFO] Batch_size: 32
[INFO] Number of bands: 4
[INFO] Dropout rate: 0.5
[INFO] Gamma learn

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 
Learning rate: 0.005
Train loss: 0.8101 | Train precision: 0.4656 | Train recall: 0.3957 | Train f1score: 0.4123 | Train acc: 0.5163 | Train kappa: 0.1133 
Val loss: 13.3673 | Val precision: 0.3671 | Val recall: 0.4041 | Val f1score: 0.1335 | Val acc: 0.1377 | Val kappa: 0.0661 

Epoch: 2 
Learning rate: 0.0025
Train loss: 0.6914 | Train precision: 0.6399 | Train recall: 0.6080 | Train f1score: 0.6226 | Train acc: 0.6189 | Train kappa: 0.3543 
Val loss: 0.7169 | Val precision: 0.4099 | Val recall: 0.4196 | Val f1score: 0.4069 | Val acc: 0.6064 | Val kappa: 0.2713 

Epoch: 3 
Learning rate: 0.00125
Train loss: 0.6545 | Train precision: 0.6806 | Train recall: 0.6679 | Train f1score: 0.6737 | Train acc: 0.6383 | Train kappa: 0.4054 
Val loss: 0.6370 | Val precision: 0.6468 | Val recall: 0.6991 | Val f1score: 0.6281 | Val acc: 0.6455 | Val kappa: 0.3351 

Epoch: 4 
Learning rate: 0.000625
Train loss: 0.6393 | Train precision: 0.7043 | Train recall: 0.6960 | Train f1score: 0.7000 

learning_rate,█▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_f1_score,▁▅▆▇█████████████████████
train_loss,█▄▃▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_f1_score,▁▄▇██▇███████████████████
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,0.0
train_f1_score,0.73691
train_loss,0.60752
val_f1_score,0.70009
val_loss,0.61681


...

Check shapes:

Images train dataset: (12498, 250, 250, 4)
Labels train dataset: (12498, 1)

Images validation dataset: (1065, 250, 250, 4)
Labels validation dataset: (1065, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in train dataset: 7407
Stressed trees in train dataset: 4870
Dead trees in train dataset: 221
Healthy trees in validation dataset: 620
Stressed trees in validation dataset: 431
Dead trees in validation dataset: 14
Ratio health trees in validation dataset: 0.067157712305026
Ratio stressed trees in validation dataset: 0.06989944858903666
Ratio dead trees in validation dataset: 0.05363984674329502
--------------------------------------------------

Creating datasets for fold: 4

Creating dataloaders for fold: 4

Loaded pretrained weights for efficientnet-b7

[INFO] Fold number: 4
[INFO] Number of epochs: 50
[INFO] Batch_size: 32
[INFO] Number of bands: 4
[INFO] Dropout rate: 0.5
[INFO] Gamma learnin

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 
Learning rate: 0.005
Train loss: 0.8069 | Train precision: 0.4914 | Train recall: 0.4433 | Train f1score: 0.4605 | Train acc: 0.5370 | Train kappa: 0.1886 
Val loss: 0.9314 | Val precision: 0.4160 | Val recall: 0.3810 | Val f1score: 0.3235 | Val acc: 0.5814 | Val kappa: 0.0300 

Epoch: 2 
Learning rate: 0.0025
Train loss: 0.6894 | Train precision: 0.6321 | Train recall: 0.6094 | Train f1score: 0.6195 | Train acc: 0.6195 | Train kappa: 0.3795 
Val loss: 0.6453 | Val precision: 0.6968 | Val recall: 0.6800 | Val f1score: 0.6783 | Val acc: 0.6487 | Val kappa: 0.3744 

Epoch: 3 
Learning rate: 0.00125
Train loss: 0.6520 | Train precision: 0.6912 | Train recall: 0.6873 | Train f1score: 0.6891 | Train acc: 0.6376 | Train kappa: 0.4146 
Val loss: 0.6287 | Val precision: 0.6101 | Val recall: 0.7602 | Val f1score: 0.6618 | Val acc: 0.6525 | Val kappa: 0.3829 

Epoch: 4 
Learning rate: 0.000625
Train loss: 0.6421 | Train precision: 0.7008 | Train recall: 0.6955 | Train f1score: 0.6978 |

Epoch: 29 
Learning rate: 1.862645149230957e-11
Train loss: 0.6187 | Train precision: 0.7288 | Train recall: 0.7311 | Train f1score: 0.7298 | Train acc: 0.6632 | Train kappa: 0.4633 
Val loss: 0.5900 | Val precision: 0.7150 | Val recall: 0.7870 | Val f1score: 0.7464 | Val acc: 0.6951 | Val kappa: 0.4427 

Epoch: 30 
Learning rate: 9.313225746154785e-12
Train loss: 0.6180 | Train precision: 0.7278 | Train recall: 0.7315 | Train f1score: 0.7294 | Train acc: 0.6594 | Train kappa: 0.4545 
Val loss: 0.5889 | Val precision: 0.7141 | Val recall: 0.7856 | Val f1score: 0.7452 | Val acc: 0.6941 | Val kappa: 0.4400 

Epoch: 31 
Learning rate: 4.656612873077393e-12
Train loss: 0.6147 | Train precision: 0.7362 | Train recall: 0.7278 | Train f1score: 0.7318 | Train acc: 0.6660 | Train kappa: 0.4686 
Val loss: 0.5904 | Val precision: 0.7127 | Val recall: 0.7856 | Val f1score: 0.7445 | Val acc: 0.6913 | Val kappa: 0.4371 

Epoch: 32 
Learning rate: 2.3283064365386963e-12
Train loss: 0.6152 | Train pre

KeyboardInterrupt: 

In [ ]:
# Create a graph to visualize the training and validation loss for each fold
wandb.run.summary["graph"] = wandb.plot.line_series(
    xs="epoch",
    ys=["train_loss", "val_loss"],
    group="fold",
    xaxis="Epoch",
    yaxis="Loss",
    title="Training and Validation Loss by Fold",
)

# Create a graph to visualize the training and validation f1 score for each fold
wandb.run.summary["graph"] = wandb.plot.line_series(
    xs="epoch",
    ys=["train_f1_score", "val_f1_score"],
    group="fold",
    xaxis="Epoch",
    yaxis="Loss",
    title="Training and Validation F1-Score by Fold",
)

In [ ]:
# create test dataset
test_dataset = data_loader.CustomTestDataset(
    data = test_image_set,
    labels = test_label_set,
    class_names=class_names, 
    species = test_species_set,
    kkl = None,
    transform=transform
)

# create test dataloader
test_dataloader = DataLoader(test_dataset,
                             batch_size=batch_size,
                             persistent_workers=True,
                             pin_memory=True,
                             num_workers=NUM_WORKERS,
                             shuffle=False,
                             drop_last=True)

In [ ]:
# Setup the best model filepath
best_model_path = r"C:\Users\lwfeckesim\01_PyTorch\wze-uav\wze-uav-master\effnet_b0\01_18_epochs.pth"

# Instantiate a new instance of EffNetB0 (to load the saved state_dict() to)
unfreeze=True
best_model = models.create_effnetb0(output_shape=num_classes, unfreeze=unfreeze, dropout_rate=dropout_rate, device=device)

# Load the saved best model state_dict()
best_model.load_state_dict(torch.load(best_model_path))

In [ ]:
def make_predictions(model: torch.nn.Module, 
                     test_dataloader: torch.utils.data.DataLoader,
                     device: torch.device):
    # 1. Make predictions with trained model
    y_preds = []
    y_labels = []
    species_list = []
    test_loss, test_precision, test_recall, test_f1_score, test_acc = 0, 0, 0, 0, 0
    count = 0
    model.eval()
    with torch.inference_mode():
        for X, y, species in tqdm(test_dataloader, desc="Making predictions"):
            # Send data and targets to target device
            X, y = X.to(device), y.to(device)
            # Do the forward pass
            y_logit = model(X)
            # Turn predictions from logits -> prediction probabilities -> predictions labels
            y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
            # Put predictions on CPU for evaluation
            y_preds.append(y_pred.cpu())
            y_labels.append(y.cpu())
            species_list.append(species)
            
            #other metrics
            test_acc += ((y_pred == y).sum().item()/len(y_pred))
            y_pred_class = y_pred.detach().cpu().numpy() 
            y_class = y.detach().cpu().numpy()
            labels = np.array([0])
            test_precision += precision_score(y_class, y_pred_class, average='macro', zero_division=0, labels=[0,1,2])
            test_recall += recall_score(y_class, y_pred_class, average='macro', zero_division=0, labels=[0,1,2])
            #test_f1_score += f1_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
            
            #if count >= 1:
            #    y_set = torch.cat((y_set, y))
            #    count = count + 1
            #else:
            #    y_set = y
            #    count = count + 1
            
    test_loss = test_loss / len(test_dataloader)
    test_precision = test_precision / len(test_dataloader)
    test_recall = test_recall / len(test_dataloader)
    test_f1_score = test_f1_score / len(test_dataloader)
    #test_kappa = test_kappa / len(dataloader)
    test_acc = test_acc / len(test_dataloader)
    # Concatenate list of predictions into a tensor
    y_pred_tensor = torch.cat(y_preds)
    y_labels_tensor = torch.cat(y_labels)
    test_f1_score = f1_score(y_labels_tensor.detach().cpu().numpy(), y_pred_tensor.cpu().numpy(), average='macro', zero_division=1, labels=[0,1,2])
    
    # Print classification report
    y_true = y_labels_tensor.detach().cpu().numpy()
    report = classification_report(y_true, y_pred_tensor.cpu().numpy(), target_names=class_names)
    print(report)
    
    return y_pred_tensor, y_labels_tensor, test_loss, test_recall, test_precision, test_f1_score, test_acc, y_logit, y_pred, y, y_preds

In [ ]:
# 2. Setup confusion matrix instance and compare predictions to targets
#from wze_uav.analysis import *
y_pred_tensor, y_labels_tensor, test_loss, test_recall, test_precision, test_f1_score, test_acc, y_logit, y_pred, y, y_preds = make_predictions(model=best_model,
                                 test_dataloader=test_dataloader, 
                                 device=device)

y_labels_tensor = y_labels_tensor.detach().cpu().numpy()
y_pred_tensor = y_pred_tensor.detach().cpu().numpy()

#confmat = ConfusionMatrix(num_classes=num_classes, task='multiclass')
#confmat_tensor = confmat(preds=y_pred_tensor,
#                         target=test_labels)
labels = np.array([0,1,2])
confmat = confusion_matrix(y_labels_tensor, y_pred_tensor, labels=labels)

# 3. Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat, # matplotlib likes working with NumPy 
    class_names=class_names, # turn the row and column labels into class names
    figsize=(10, 7)
);

print(f"Test loss: {test_loss}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Test F1score: {test_f1_score}")
#print(f"Test Kappa: {test_kappa}")
print(f"Test Accuracy: {test_acc}")
print(f"Test Logits: {y_logit}")
print(f"Test Predictions: {y_pred}")
print(f"Test Labels: {y}")

In [ ]:
len(y_preds)

In [ ]:
y_set.cpu()

In [ ]:
test_dataset.labels

In [ ]:
y_preds = []
y_labels = []
labels = np.array([0,1,2])
test_loss, test_precision, test_recall, test_f1_score, test_acc = 0, 0, 0, 0, 0
count = 0
model.eval()
with torch.inference_mode():
    for X, y in tqdm(test_dataloader, desc="Making predictions"):
        # Send data and targets to target device
        X, y = X.to(device), y.to(device)
        # Do the forward pass
        y_logit = model(X)
        # Turn predictions from logits -> prediction probabilities -> predictions labels
        y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
        # Put predictions on CPU for evaluation
        y_preds.append(y_pred.cpu())
        y_labels.append(y.cpu())
        
        #other metrics
        test_acc += ((y_pred == y).sum().item()/len(y_pred))
        y_pred_class = y_pred.detach().cpu().numpy() 
        y_class = y.detach().cpu().numpy()
        test_precision += precision_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
        test_recall += recall_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
        #test_f1_score += f1_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
        
        #if count >= 1:
        #    y_set = torch.cat((y_set, y))
        #    count = count + 1
        #else:
        #    y_set = y
        #    count = count + 1
        
test_loss = test_loss / len(test_dataloader)
test_precision = test_precision / len(test_dataloader)
test_recall = test_recall / len(test_dataloader)
#test_f1_score = test_f1_score / len(test_dataloader)
#test_kappa = test_kappa / len(dataloader)
test_acc = test_acc / len(test_dataloader)
# Concatenate list of predictions into a tensor
y_pred_tensor = torch.cat(y_preds)
test_f1_score = f1_score(y_set.detach().cpu().numpy(), y_pred_tensor.cpu().numpy(), average='macro', zero_division=0, labels=[0,1,2])

# Print classification report
y_true = y_set.detach().cpu().numpy()
report = classification_report(y_true, y_pred_tensor.cpu().numpy(), target_names=class_names)
print(report)

In [ ]:
test_f1_score

In [ ]:
make = (y_class == y_pred_class)
make

In [ ]:
torch.softmax(y_logit, dim=1).argmax(dim=1)

In [ ]:
y

In [ ]:
test = (y_pred == y).sum().item()/16

In [ ]:
test

In [ ]:
 y_pred_class = y_pred.detach().cpu().numpy() 

In [ ]:
y_pred_class